In [ ]:
import tensorflow.keras as tfk
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
from tensorflow.keras.utils import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tqdm import *
import pickle
import re

Download cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = tfk.datasets.cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
x_train = x_train / 255.0
x_test = x_test / 255.0

Model

In [ ]:
class SignGate(Layer):
    def __init__(self, **kwargs):
        super(SignGate, self).__init__(**kwargs)

    def build(self, input_shape):
        super(SignGate, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        output = K.sign(K.relu(x))

        return output

    def compute_output_shape(self, input_shape):
        return input_shape

def getNetwork1Layers(model):
    pattern = re.compile("^n1_*")
    layer_list  = []

    for layer in model.layers:
        if pattern.match(layer.name):
            layer_list.append(layer)

    return layer_list

def getNetwork2Layers(model):
    pattern = re.compile("^G[0-9]*$")
    layer_list  = []

    for layer in model.layers:
        if pattern.match(layer.name):
            layer_list.append(layer)

    return layer_list

def freezeWeights(layers):
    for layer in layers:
        print(layer.name)
        layer.trainable = False

img_wid = 32
def getConv4Relu():
    inputs = Input(shape = (img_wid, img_wid, 3))
    
    C1 = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n1_c1",
                           activation = 'relu')(inputs)
    C2 = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n1_c2",
                           activation = 'relu')(C1)
    C3 = Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', name = "n1_c3",
                           activation = 'relu')(C2)
    C4 = Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', name = "n1_c4",
                           activation = 'relu')(C3)

    G1 = GlobalAveragePooling2D()(C4)
    F1 = Flatten()(G1)
    D1 = Dense(units = 256, activation = 'relu', name = "n1_d1")(F1)
    D2 = Dense(units = 256, activation = 'relu', name = "n1_d2")(D1)

    #Output
    outputs = Dense(units = 10, activation = 'softmax', name = "n1_o1")(D2)

    model = tfk.Model(inputs = inputs, outputs = outputs)

    return model

def getConv4Galu():
    inputs = Input(shape = (img_wid, img_wid, 3))
    #V1
    C1 = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n1_c1",
                           activation = 'relu')(inputs)
    C2 = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n1_c2",
                           activation = 'relu')(C1)
    C3 = Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', name = "n1_c3",
                           activation = 'relu')(C2)
    C4 = Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', name = "n1_c4",
                           activation = 'relu')(C3)
    G1 = GlobalAveragePooling2D()(C4)
    F1 = Flatten()(G1)
    D1 = Dense(units = 256, activation = 'relu', name = "n1_d1")(F1)
    D2 = Dense(units = 256, activation = 'relu', name = "n1_d2")(D1)

    A1 = SignGate()(C1)
    A2 = SignGate()(C2)
    A3 = SignGate()(C3)
    A4 = SignGate()(C4)
    A5 = SignGate()(D1)
    A6 = SignGate()(D2)
    C1_G = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n2_c1",
                           activation = 'linear')(inputs)
    C1_G = Multiply()([A1, C1_G])

    C2_G = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', name = "n2_c2", 
                           activation = 'linear')(C1_G)
    C2_G = Multiply()([A2, C2_G])

    #V2
    C3_G = Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', name = "n2_c3",
                           activation = 'linear')(C2_G)
    C3_G = Multiply()([A3, C3_G])

    C4_G = Conv2D(filters = 128, kernel_size = (3, 3),  padding = 'same', name = "n2_c4",
                           activation = 'linear')(C3_G)
    C4_G = Multiply()([A4, C4_G])

    G1_G = GlobalAveragePooling2D()(C4_G)
    F1_G = Flatten()(G1_G)

    D1_G = Dense(units = 256, activation = 'linear', name = "n2_d1")(F1_G)
    D1_G = Multiply()([A5, D1_G])

    D2_G = Dense(units = 256, activation = 'linear', name = "n2_d2")(D1_G)
    D2_G = Multiply()([A6, D2_G])

    outputs = Dense(units = 10, activation = 'softmax', name = "n2_output")(D2_G)

    model = tfk.Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
lr = 3e-4
loss = tfk.losses.categorical_crossentropy
opt = tfk.optimizers.Adam
batch_size = 32
num_exp = 3

history_relu = {'acc':[], 'val_acc':[], 'loss': [], 'val_loss': [], 'train_margin': [], 'test_margin': []}
history_galu_learned = {'acc':[], 'val_acc':[], 'loss': [], 'val_loss': [], 'train_margin': [], 'test_margin': []}

Train

In [ ]:
for exp_i in range(num_exp):
    print("_____________EXP:{}____________".format(exp_i+1))

    model_relu = getConv4Relu()
    model_relu.compile(loss = loss, optimizer = opt(lr), metrics = ['acc'])

    filepath="weights.best.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    relu_temp_acc = []
    relu_temp_val = []
    galu_temp_acc = []
    galu_temp_val = []

    model_galu = getConv4Galu()
    layers_galu = getNetwork1Layers(model_galu)
    for layer in layers_galu:
        layer.trainable = False
    model_galu.compile(loss = loss, optimizer = opt(lr), metrics = ['acc'])

    history_g = model_galu.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 0,
                    batch_size=batch_size, epochs= 80)
    
    galu_temp_acc.append(np.max(history_g.history['acc']))
    galu_temp_val.append(np.max(history_g.history['val_acc']))
    
    for epoch_i in range(6):
        history_r = model_relu.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 0,
                                batch_size=batch_size, epochs= 10, callbacks = callbacks_list)
        relu_temp_acc.append(np.max(history_r.history['acc']))
        relu_temp_val.append(np.max(history_r.history['val_acc']))
    
        print("ReLU: MAX ACC = {}, MAX VAL ACC = {}".format(np.max(history_r.history['acc']), 
                                                            np.max(history_r.history['val_acc'])))
        model_relu1 = getConv4Relu()
        model_relu1.compile(loss = loss, optimizer = opt(lr), metrics = ['acc'])
        model_relu1.load_weights("weights.best.hdf5")    #load weights of relu
        
        model_galu_learned = getConv4Galu()
        layers_relu = getNetwork1Layers(model_relu1)[:-1]
        layers_galu_learned = getNetwork1Layers(model_galu_learned)
        
        for layer in layers_galu_learned:
            layer.trainable = False

        model_galu_learned.compile(loss = loss, optimizer = opt(lr), metrics = ['acc'])

        for layer1, layer2 in zip(layers_galu_learned, layers_relu): 
            layer1.set_weights(layer2.get_weights())       #copy weights of trained relu network in gating network

        history_g = model_galu_learned.fit(x_train, y_train, validation_data = (x_test, y_test), verbose = 0,
                                        batch_size=batch_size, epochs= 80)
        galu_temp_acc.append(np.max(history_g.history['acc']))
        galu_temp_val.append(np.max(history_g.history['val_acc']))
        print("GaLU Learned: MAX ACC = {}, MAX VAL ACC = {}".format(np.max(history_g.history['acc']), 
                                                        np.max(history_g.history['val_acc'])))
        print("----------------------------------------------------------------------------------")
    history_galu_learned['acc'].append(galu_temp_acc)
    history_galu_learned['val_acc'].append(galu_temp_val)
    history_relu['acc'].append(relu_temp_acc)
    history_relu['val_acc'].append(relu_temp_val)

In [ ]:
 print("ReLU: max_acc = {:.4f}, mean_max_val_acc = {:.4f}, std_max_val_acc = {:.4f}".format(
                                                    np.mean(np.max(history_relu['acc'], axis = 1)), 
                                                    np.mean(np.max(history_relu['val_acc'], axis = 1)),
                                                    np.std(np.max(history_relu['val_acc'], axis = 1))))

print("GaLU Learned: max_acc = {:.4f}, mean_max_val_acc = {:.4f}, std_max_val_acc = {:.4f}".format(
                                                    np.mean(np.max(history_galu_learned['acc'], axis = 1)), 
                                                    np.mean(np.max(history_galu_learned['val_acc'], axis = 1)),
                                                    np.std(np.max(history_galu_learned['val_acc'], axis = 1))))

- Save Results

In [ ]:
file = open("h_conv4_multi_step_copy_relu", 'wb')
pickle.dump(history_relu, file)

file = open("h_conv4_multi_step_copy_galu_learned", 'wb')
pickle.dump(history_galu_learned, file)

In [ ]:
flnpf = np.mean(np.array(history_galu_learned['val_acc']), axis = 0)
print(flnpf.shape)

- Plot Results

In [ ]:
stage = np.arange(7).tolist()
flnpf = flnpf*100
cntk = [77.43 for _ in range(25)]
relu = [80.32 for _ in range(25)]
frnpf = [flnpf[0] for _ in range(25)]
temp_x =  np.arange(0, 6.01, 0.25)

In [ ]:
plt.plot(stage, flnpf, marker = 'o', label = 'FLNPF')
plt.plot(temp_x, cntk, '--', label = ' CNTK Arora et al. [2019]')
plt.plot(temp_x, relu, '--', label = 'RELU', marker = '>')
plt.plot(temp_x, frnpf, '--', label = 'FRNPF', marker = '*')

yticks = (flnpf[:-4].tolist() + [flnpf[-1]]+[cntk[0]]+[relu[0]])

plt.legend(fontsize = 12)
plt.xlabel('Stage', fontsize = 16)
plt.ylabel('Test Accuracy(%)', fontsize = 16)
plt.yticks(yticks)
plt.show()